# SQL Avancé

## Révisions

\- Schémas

\- Création de table

## Schémas

Un schéma permet de grouper des tables de même nature pour gérer les permissions plus efficacement.

  

Traditionnellement on va avoir deux schémas différents :

\- data --\> Lecture écriture

\- ref --\> Lecture seule

In [ ]:
IF (EXISTS (
        SELECT [name]
        FROM [master].[sys].[databases]
        WHERE [name] = 'Forum'))
            DROP DATABASE [Forum]

CREATE DATABASE [Forum]

GO

USE [Forum]

In [ ]:
CREATE SCHEMA [data]
GO
CREATE SCHEMA [ref]
GO